# Guardian - Iteration 3

## Import Packages

In [1]:
# load .env File in Environment
from dotenv import load_dotenv
# operating system interfaces - get Environment-variables
import os
# Connect to API
import requests
# create Pandas Data Frame
import pandas as pd
import datetime as dt
from functools import wraps
load_dotenv()

True

### Set Params

In [2]:
iteration = 3

content_url = "https://content.guardianapis.com/search"
all_content_params = {
    "page-size": "50",
    "show-fields": "all",
    "show-tags": "all",
    "from-date": "2021-03-01",
    "to-date": "2021-08-01",
    "order-by": "newest",
    "api-key": os.getenv("GUARDIAN_API_KEY2"),
}


In [3]:
def log_step(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        tic = dt.datetime.now()
        result = func(*args, **kwargs)
        time_taken = str(dt.datetime.now() - tic)
        print(f"{func.__name__}:\n shape={result.shape} took {time_taken}s\n")
        return result

    return wrapper

### Create DataFrame function


In [4]:
def get_df(url: str, params: dict):
        all_results = []
        current_page = 1
        total_pages = 1
        while current_page <= total_pages:
                tic = dt.datetime.now()
                params["page"] = current_page
                try:
                        r = requests.get(url, params)
                        all_results = all_results + r.json()["response"]["results"]
                        r.raise_for_status()
                except Exception as err:
                        SystemExit(err)
                if current_page == 1:
                        print("---- API STATUS ---- ")
                        print("status",  r.json()["response"]["status"])
                        total_pages = r.json()['response']['pages']
                        print("URL: ", r.url)
                        print("status",  r.json()["response"]["status"])
                        print("total",  r.json()["response"]["total"])
                        print("startIndex",  r.json()["response"]["startIndex"])
                        print("pageSize",  r.json()["response"]["pageSize"])
                        print("pages",  r.json()["response"]["pages"])
                        print("orderBy",  r.json()["response"]["orderBy"])
                        print("---- RUNTIME STATUS ---- ")

                time_taken = str(dt.datetime.now() - tic)
                print(f"({current_page}/{total_pages}) in {time_taken}s")
                        
                current_page += 1

        return pd.DataFrame(all_results)

### API Request

In [5]:
init_df = get_df(content_url, all_content_params)

---- API STATUS ---- 
status ok
URL:  https://content.guardianapis.com/search?page-size=50&show-fields=all&show-tags=all&from-date=2021-03-01&to-date=2021-08-01&order-by=newest&api-key=934fc87f-0792-4733-b5ed-59d5aa136db0&page=1
status ok
total 32701
startIndex 1
pageSize 50
pages 655
orderBy newest
---- RUNTIME STATUS ---- 
(1/655) in 0:00:02.589941s
(2/655) in 0:00:04.010012s
(3/655) in 0:00:02.604894s
(4/655) in 0:00:01.532496s
(5/655) in 0:00:02.258086s
(6/655) in 0:00:03.270420s
(7/655) in 0:00:03.080278s
(8/655) in 0:00:02.127847s
(9/655) in 0:00:01.986589s
(10/655) in 0:00:01.581121s
(11/655) in 0:00:02.228960s
(12/655) in 0:00:01.993420s
(13/655) in 0:00:12.462177s
(14/655) in 0:00:06.351910s
(15/655) in 0:00:04.891889s
(16/655) in 0:00:01.580827s
(17/655) in 0:00:05.764226s
(18/655) in 0:00:01.878755s
(19/655) in 0:00:04.210476s
(20/655) in 0:00:02.276649s
(21/655) in 0:00:07.026712s
(22/655) in 0:00:01.138399s
(23/655) in 0:00:01.107059s
(24/655) in 0:00:02.663901s
(25/655) i

In [6]:
@log_step
def init_pipeline(df):
    return df.copy()

@log_step
def unfold_columns(df):
    dict_cols = ["fields"] 
    for col in dict_cols:
        new_df = pd.DataFrame()
        new_df[col] = df[col]#.apply(lambda x: eval(x))
        add_cols_df = pd.json_normalize(new_df[col])
        df = pd.concat([df, add_cols_df], axis=1)
    
    # Tags extraction
    
    #df["tags"] = df["tags"].apply(lambda x: eval(x))
    df['tagWebTitle'] = df['tags'].map(lambda x:[i['webTitle'] for i in x])
    df['tagId'] = df['tags'].map(lambda x:[i['id'] for i in x])
    df = df.drop(columns="tags")
    return df.drop(columns=dict_cols)
@log_step
def drop_columns(df):
    df = df.drop(columns=[
        "body",
        "webUrl",
        "apiUrl",
        "sectionId",
        "pillarId",
        "showAffiliateLinks",
        "bylineHtml",
        "standfirst",
        "shouldHideReaderRevenue",
        "isInappropriateForSponsorship",
        "shortUrl",
        "isPremoderated",
        "charCount",
        "shouldHideAdverts",
        "showInRelatedContent",
        "lang",
        "main",
        "thumbnail"
        
    ])
    return df

@log_step
def create_csv(df):
    df.to_csv(f"../../data/partial/articles_partial_{iteration}.csv.zip", compression="zip")
    return df

In [7]:
articles = (
    init_df
        .pipe(init_pipeline)
        .pipe(unfold_columns)
        .pipe(drop_columns)
        .pipe(create_csv)
)

init_pipeline:
 shape=(32701, 13) took 0:00:00.010500s

unfold_columns:
 shape=(32701, 48) took 0:00:08.658574s

drop_columns:
 shape=(32701, 30) took 0:00:00.024476s

create_csv:
 shape=(32701, 30) took 0:00:28.940077s

